In [19]:
# %pip install -q ydata-profiling xgboost
# %pip install -q scikit-learn pandas numpy matplotlib seaborn
# ------------------------------------------------------------

# ------------------------- Imports ---------------------------
import os, sys, gc, json, zipfile, math, warnings
from datetime import datetime
from pathlib import Path
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    roc_auc_score, roc_curve, average_precision_score,
    precision_recall_fscore_support, accuracy_score,
    confusion_matrix
)
from sklearn.impute import SimpleImputer

warnings.filterwarnings('ignore')
np.random.seed(42)
pd.set_option('display.max_columns', 120)

print('Libs carregadas. pandas =', pd.__version__)

# ------------------- Parâmetros/Flags -----------------------
# Você baixará a base manualmente. Escolha UMA das opções de entrada abaixo:
INPUT_MODE = 'UPLOAD'   # 'UPLOAD' ou 'DRIVE'

GENERATE_PROFILE = False   # True: gera relatório HTML com ydata-profiling (pesado)
SAMPLE_ROWS = 200_000      # Amostragem para performance (None para usar tudo)

DATA_PATH = Path('data')
DATA_PATH.mkdir(exist_ok=True)

# ----------------- Funções utilitárias ----------------------
def build_data_dictionary(df: pd.DataFrame) -> pd.DataFrame:
    """Dicionário de dados básico para governança/qualidade."""
    dd = []
    for c in df.columns:
        n_null = int(df[c].isna().sum())
        pct_null = float(n_null/len(df)) * 100 if len(df) else 0.0
        n_unique = int(df[c].nunique(dropna=True))
        example = df[c].dropna().iloc[0] if df[c].dropna().shape[0] else None
        dd.append({
            'column': c,
            'dtype': str(df[c].dtype),
            'n_missing': n_null,
            'pct_missing': round(pct_null, 2),
            'n_unique': n_unique,
            'example': example
        })
    return pd.DataFrame(dd).sort_values(['pct_missing','column'], ascending=[False, True]).reset_index(drop=True)

def _read_lending_club_csv(path: Path) -> pd.DataFrame:
    """
    Lê CSV do Lending Club:
    - Muitos LoanStats têm 1ª linha de metadados -> skiprows=1
    - Remove linhas completamente vazias no final
    """
    df = pd.read_csv(path, low_memory=False)
    df = df.dropna(how='all')
    if 'loan_status' not in df.columns:
        raise ValueError('Coluna loan_status não encontrada. Use LoanStats_*.csv válido.')
    return df

def load_lending_club(raw_file: Path) -> pd.DataFrame:
    """Lê CSV ou ZIP contendo CSV do Lending Club."""
    if raw_file is None or not Path(raw_file).exists():
        raise FileNotFoundError('Arquivo não encontrado. Faça upload ou aponte o caminho no DRIVE_FILE.')
    raw_file = Path(raw_file)
    if raw_file.suffix == '.zip':
        with zipfile.ZipFile(raw_file, 'r') as z:
            csv_names = [n for n in z.namelist() if n.lower().endswith('.csv')]
            if not csv_names:
                raise ValueError('ZIP não contém CSVs. Verifique o conteúdo.')
            # Se houver mais de um csv, pegue o maior (geralmente a base principal)
            csv_names_sorted = sorted(csv_names, key=lambda n: z.getinfo(n).file_size, reverse=True)
            target_csv = csv_names_sorted[0]
            print('Lendo CSV dentro do zip:', target_csv)
            with z.open(target_csv) as f:
                df = pd.read_csv(f, low_memory=False)
                df = df.dropna(how='all')
    else:
        df = _read_lending_club_csv(raw_file)
    return df

def ks_statistic(y_true, y_prob):
    """KS e threshold correspondente (máx TPR-FPR)."""
    fpr, tpr, thr = roc_curve(y_true, y_prob)
    ks = np.max(tpr - fpr)
    ix = np.argmax(tpr - fpr)
    return float(ks), float(thr[ix])

def top_decile_lift(y_true, y_prob):
    """Lift e capture rate no top decil de score."""
    y_true = np.asarray(y_true)
    y_prob = np.asarray(y_prob)
    cutoff = np.quantile(y_prob, 0.9)
    top_mask = y_prob >= cutoff
    rate_overall = y_true.mean() if len(y_true) else 0.0
    rate_top = y_true[top_mask].mean() if top_mask.any() else 0.0
    lift = (rate_top / rate_overall) if rate_overall > 0 else np.nan
    capture = y_true[top_mask].sum() / y_true.sum() if y_true.sum() > 0 else np.nan
    return float(lift), float(capture)

# --------------- Entrada de dados: Upload ou Drive ----------
RAW_FILE = None

if INPUT_MODE.upper() == 'UPLOAD':
    print('>> Modo UPLOAD: usando arquivo LoanStats.csv existente')
    src = Path('/content/LoanStats.csv')  # Path to the existing file
    if not src.exists():
        raise FileNotFoundError(f'Não encontrei o arquivo: {src}. Faça upload ou aponte o caminho no DRIVE_FILE.')
    dst = DATA_PATH / src.name
    import shutil; shutil.copy(str(src), str(dst))
    RAW_FILE = dst
    print('Arquivo copiado para', RAW_FILE)

elif INPUT_MODE.upper() == 'DRIVE':
    print('>> Modo DRIVE: montando o Google Drive...')
    try:
        from google.colab import drive
        drive.mount('/content/drive')
        RAW_FILE = Path(DRIVE_FILE)
        if not RAW_FILE.exists():
            raise FileNotFoundError(f'Não encontrei o arquivo no Drive: {RAW_FILE}')
        # Copia para data/ para padronizar
        dst = DATA_PATH / RAW_FILE.name
        import shutil; shutil.copy(str(RAW_FILE), str(dst))
        RAW_FILE = dst
        print('Arquivo copiado para', RAW_FILE)
    except Exception as e:
        raise RuntimeError('Falha ao acessar o Drive. Erro: ' + str(e))
else:
    raise ValueError("INPUT_MODE inválido. Use 'UPLOAD' ou 'DRIVE'.")

print('Arquivo detectado:', RAW_FILE)

# ------------------ Leitura e dicionário --------------------
df_raw = load_lending_club(RAW_FILE)
print('Shape bruto:', df_raw.shape)

data_dict_raw = build_data_dictionary(df_raw)
data_dict_raw.to_csv(DATA_PATH/'data_dictionary.csv', index=False)
print('Dicionário (raw) salvo em', DATA_PATH/'data_dictionary.csv')

# ----------- (Opcional) Perfil de qualidade (HTML) ----------
if GENERATE_PROFILE:
    try:
        from ydata_profiling import ProfileReport
        profile = ProfileReport(df_raw, title='Data Quality – Lending Club', explorative=True)
        profile_path = DATA_PATH/'data_quality_profile.html'
        profile.to_file(profile_path)
        print('Relatório de qualidade salvo em', profile_path)
    except Exception as e:
        print('Falha ao gerar perfil de qualidade (ydata-profiling):', e)
else:
    print('Pulando geração do perfil para rodar mais leve.')

# --------- Preprocessamento e feature engineering -----------
DEFAULT_STATUSES = {
    'Charged Off','Default','Late (31-120 days)','Late (16-30 days)',
    'Does not meet the credit policy. Status:Charged Off'
}
NONDEFAULT_STATUSES = {
    'Fully Paid', 'Does not meet the credit policy. Status:Fully Paid'
}

def preprocess(df: pd.DataFrame, sample_rows: int = SAMPLE_ROWS):
    """
    - Filtra statuses bem definidos e cria alvo binário 'default'.
    - Converte percentuais: int_rate, revol_util.
    - Converte datas: issue_d, earliest_cr_line; cria 'credit_history_months'.
    - Extrai 'term_months' a partir de 'term'.
    - Cria 'log_annual_inc' (transformação log).
    - Seleciona features comuns ao crédito.
    - Amostra linhas para performance, se configurado.
    """
    df = df.copy()
    df = df[df['loan_status'].isin(DEFAULT_STATUSES.union(NONDEFAULT_STATUSES))].copy()
    df['default'] = df['loan_status'].apply(lambda s: 1 if s in DEFAULT_STATUSES else 0)

    for col in ['int_rate','revol_util']:
        if col in df.columns:
            df[col] = (df[col].astype(str).str.replace('%','', regex=False)
                                 .replace('nan', np.nan).astype(float))

    for col in ['issue_d','earliest_cr_line']:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], format='%b-%Y', errors='coerce')

    if 'issue_d' in df.columns and 'earliest_cr_line' in df.columns:
        chm = (
            (df['issue_d'].dt.year - df['earliest_cr_line'].dt.year) * 12
            + (df['issue_d'].dt.month - df['earliest_cr_line'].dt.month)
        )
        df['credit_history_months'] = chm.astype('float')

    if 'term' in df.columns:
        df['term_months'] = df['term'].astype(str).str.extract(r'(\d+)').astype(float)

    if 'annual_inc' in df.columns:
        df['log_annual_inc'] = np.log1p(pd.to_numeric(df['annual_inc'], errors='coerce'))

    candidate_features = [
        'loan_amnt','funded_amnt','int_rate','installment','grade','sub_grade',
        'emp_length','home_ownership','annual_inc','verification_status','purpose',
        'dti','delinq_2yrs','inq_last_6mths','open_acc','pub_rec','revol_bal','revol_util',
        'total_acc','issue_d','earliest_cr_line','term','credit_history_months','term_months',
        'log_annual_inc'
    ]
    features = [c for c in candidate_features if c in df.columns]
    base_cols = features + ['default']
    df = df[base_cols].copy()

    all_null_cols = [c for c in df.columns if df[c].isna().all()]
    if all_null_cols:
        df = df.drop(columns=all_null_cols)

    if sample_rows and len(df) > sample_rows:
        df = df.sample(n=sample_rows, random_state=42).reset_index(drop=True)

    return df

df = preprocess(df_raw)
print('Shape após preprocessamento:', df.shape)

# --------------- Split, pipeline e treino -------------------
target = 'default'
feature_cols = [c for c in df.columns if c != target]
num_cols = [c for c in feature_cols if pd.api.types.is_numeric_dtype(df[c])]
cat_cols = [c for c in feature_cols if c not in num_cols]
print('Numéricas:', len(num_cols), '| Categóricas:', len(cat_cols))

X = df[feature_cols].copy()
y = df[target].astype(int).copy()

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

numeric_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='mean')),('scaler', StandardScaler(with_mean=False))])
categorical_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='constant', fill_value='missing')),('ohe', OneHotEncoder(handle_unknown='ignore', sparse_output=True))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, num_cols),
        ('cat', categorical_transformer, cat_cols)
    ]
)

# Modelo baseline: Logistic Regression
log_reg = Pipeline(steps=[
    ('prep', preprocessor),
    ('clf', LogisticRegression(max_iter=200, solver='saga'))
])
log_reg.fit(X_train, y_train)
proba_lr = log_reg.predict_proba(X_test)[:, 1]
auc_lr = roc_auc_score(y_test, proba_lr)
print(f'AUC Logistic Regression: {auc_lr:.4f}')

# Modelo opcional: XGBoost (se instalado)
proba_xgb, auc_xgb, xgb = None, None, None
try:
    from xgboost import XGBClassifier
    xgb = Pipeline(steps=[
        ('prep', preprocessor),
        ('clf', XGBClassifier(
            n_estimators=300,
            max_depth=5,
            learning_rate=0.08,
            subsample=0.8,
            colsample_bytree=0.8,
            eval_metric='auc',
            tree_method='hist'
        ))
    ])
    xgb.fit(X_train, y_train)
    proba_xgb = xgb.predict_proba(X_test)[:, 1]
    auc_xgb = roc_auc_score(y_test, proba_xgb)
    print(f'AUC XGBoost: {auc_xgb:.4f}')
except Exception as e:
    print('XGBoost indisponível/erro. Usando apenas Logistic Regression. Detalhe:', e)

# Seleciona o melhor por AUC
if proba_xgb is not None and auc_xgb >= auc_lr:
    best_model_name, best_model, y_score, auc_best = 'XGBoost', xgb, proba_xgb, auc_xgb
else:
    best_model_name, best_model, y_score, auc_best = 'LogisticRegression', log_reg, proba_lr, auc_lr

print('Melhor modelo:', best_model_name, '| AUC:', round(float(auc_best), 4))

# ---------------------- Avaliação ---------------------------
auc = float(roc_auc_score(y_test, y_score))
gini = float(2 * auc - 1)
ks, thr_opt = ks_statistic(y_test.values, y_score)
pr_auc = float(average_precision_score(y_test, y_score))

y_pred_opt = (y_score >= thr_opt).astype(int)
precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred_opt, average='binary', zero_division=0)
acc = accuracy_score(y_test, y_pred_opt)
cm = confusion_matrix(y_test, y_pred_opt)

lift_top, capture_top = top_decile_lift(y_test.values, y_score)

print({
    'AUC': round(auc,4), 'Gini': round(gini,4), 'KS': round(ks,4), 'PR_AUC': round(pr_auc,4),
    'thr_opt': round(thr_opt,4), 'precision': round(float(precision),4),
    'recall': round(float(recall),4), 'f1': round(float(f1),4), 'acc': round(float(acc),4),
    'lift_top_decile': round(lift_top,4), 'capture_top_decile': round(capture_top,4)
})
print('Matriz de confusão:\n', cm)

# ----------------- Exportação para Power BI -----------------
# Score no conjunto de teste
X_test_scored = X_test.reset_index(drop=True).copy()
y_test_arr = y_test.reset_index(drop=True)
X_test_scored['y_true'] = y_test_arr
X_test_scored['y_score'] = y_score
X_test_scored['y_pred_opt'] = (y_score >= thr_opt).astype(int)

# Cria ID sintético se não houver
if not any(c in X_test_scored.columns for c in ['id','member_id','loan_id']):
    X_test_scored.insert(0, 'row_id', np.arange(len(X_test_scored)))

X_test_scored.to_csv(DATA_PATH/'loan_scored.csv', index=False)
print('Arquivo de dados salvo em', DATA_PATH/'loan_scored.csv')

# Qualidade (pós-preprocess) — para painel de governança
dq = build_data_dictionary(df)
dq.to_csv(DATA_PATH/'data_quality_summary.csv', index=False)
print('Qualidade salva em', DATA_PATH/'data_quality_summary.csv')

# Métricas globais — para painel de performance
run_ts = datetime.utcnow().strftime('%Y-%m-%d %H:%M:%SZ')
metrics = pd.DataFrame([{
    'run_timestamp_utc': run_ts,
    'model': best_model_name,
    'train_rows': int(len(X_train)),
    'test_rows': int(len(X_test)),
    'features_num': int(len(num_cols)),
    'features_cat': int(len(cat_cols)),
    'default_rate_test': float(y_test.mean()),
    'auc': float(auc),
    'gini': float(gini),
    'ks': float(ks),
    'pr_auc': float(pr_auc),
    'thr_opt': float(thr_opt),
    'precision_opt': float(precision),
    'recall_opt': float(recall),
    'f1_opt': float(f1),
    'accuracy_opt': float(acc),
    'lift_top_decile': float(lift_top),
    'capture_top_decile': float(capture_top)
}])
metrics.to_csv(DATA_PATH/'model_metrics.csv', index=False)
print('Métricas salvas em', DATA_PATH/'model_metrics.csv')

# Tabela única de KPIs — para a página principal do dashboard
dashboard_table = metrics.copy()
mean_missing_pct = dq['pct_missing'].mean() if not dq.empty else np.nan
cols_high_missing = int((dq['pct_missing'] > 20).sum()) if not dq.empty else 0

dashboard_table['avg_missing_pct'] = float(mean_missing_pct) if not np.isnan(mean_missing_pct) else np.nan
dashboard_table['cols_missing_gt20pct'] = int(cols_high_missing)
dashboard_table['dataset_rows_after_preprocess'] = int(len(df))
dashboard_table['dataset_cols_after_preprocess'] = int(df.shape[1])
dashboard_table.to_csv(DATA_PATH/'dashboard_table.csv', index=False)
print('Tabela final para dashboard salva em', DATA_PATH/'dashboard_table.csv')

print('\n✅ Pipeline concluído.')
print('Arquivos para o Power BI:')
for f in ['loan_scored.csv','model_metrics.csv','data_quality_summary.csv','dashboard_table.csv']:
    print(' -', DATA_PATH/f)

Libs carregadas. pandas = 2.2.2
>> Modo UPLOAD: usando arquivo LoanStats.csv existente
Arquivo copiado para data/LoanStats.csv
Arquivo detectado: data/LoanStats.csv
Shape bruto: (42535, 144)
Dicionário (raw) salvo em data/data_dictionary.csv
Pulando geração do perfil para rodar mais leve.
Shape após preprocessamento: (42535, 23)
Numéricas: 15 | Categóricas: 7
AUC Logistic Regression: 0.7025
AUC XGBoost: 0.6978
Melhor modelo: LogisticRegression | AUC: 0.7025
{'AUC': 0.7025, 'Gini': 0.4049, 'KS': 0.3022, 'PR_AUC': 0.2963, 'thr_opt': 0.1456, 'precision': 0.2415, 'recall': 0.6858, 'f1': 0.3572, 'acc': 0.6269, 'lift_top_decile': 2.2543, 'capture_top_decile': 0.2255}
Matriz de confusão:
 [[4451 2770]
 [ 404  882]]
Arquivo de dados salvo em data/loan_scored.csv
Qualidade salva em data/data_quality_summary.csv
Métricas salvas em data/model_metrics.csv
Tabela final para dashboard salva em data/dashboard_table.csv

✅ Pipeline concluído.
Arquivos para o Power BI:
 - data/loan_scored.csv
 - data/m